In [3]:
import numpy as np
import librosa

def wav2mfcc(file_path,max_pad_len = 11):
    wave,sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)), mode='constant')
    return mfcc

In [4]:
import os
DATA_PATH = "./DeadSimpleSpeechRecognizer-master/data/"

def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0,len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [5]:
#convert into np array
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels, _, _ = get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
        np.save(label+'.npy', mfcc_vectors)

In [6]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio = 0.8, random_state = 42):
    labels, indices, _ = get_labels(DATA_PATH)
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X,x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
    assert X.shape[0] == len(y)
    return train_test_split(X,y, test_size = (1-split_ratio), random_state=random_state, shuffle=True)

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

Using TensorFlow backend.


In [8]:
save_data_to_array()

In [9]:
X_train, X_test, y_train, y_test = get_train_test()

In [10]:
X_train = X_train.reshape(X_train.shape[0],20,11,1)
X_test = X_test.reshape(X_test.shape[0],20,11,1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [11]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2,2), activation = 'relu', input_shape=(20,11,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    model.compile(loss = keras.losses.categorical_crossentropy,
                 optimizer = keras.optimizers.Adadelta(),
                 metrics = ['accuracy'])
    return model

In [12]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size = 100, epochs = 300, verbose = 1, validation_data = (X_test, y_test_hot))

Train on 4150 samples, validate on 1038 samples
Epoch 1/300
4150/4150 [==============================] - 1s 163us/step - loss: 5.6401 - acc: 0.4814 - val_loss: 2.3477 - val_acc: 0.6561
Epoch 2/300
4150/4150 [==============================] - 0s 86us/step - loss: 3.0938 - acc: 0.6383 - val_loss: 2.2323 - val_acc: 0.6869
Epoch 3/300
4150/4150 [==============================] - 0s 73us/step - loss: 2.1439 - acc: 0.7063 - val_loss: 2.2073 - val_acc: 0.6869
Epoch 4/300
4150/4150 [==============================] - 0s 75us/step - loss: 1.5867 - acc: 0.7516 - val_loss: 0.9088 - val_acc: 0.8121
Epoch 5/300
4150/4150 [==============================] - 0s 71us/step - loss: 1.3010 - acc: 0.7699 - val_loss: 1.1767 - val_acc: 0.7630
Epoch 6/300
4150/4150 [==============================] - 0s 73us/step - loss: 1.0590 - acc: 0.7769 - val_loss: 0.6502 - val_acc: 0.8372
Epoch 7/300
4150/4150 [==============================] - 0s 72us/step - loss: 0.9390 - acc: 0.7858 - val_loss: 0.7119 - val_acc: 0.8112

4150/4150 [==============================] - 0s 69us/step - loss: 0.2746 - acc: 0.8959 - val_loss: 0.3163 - val_acc: 0.8931
Epoch 61/300
4150/4150 [==============================] - 0s 68us/step - loss: 0.2800 - acc: 0.8983 - val_loss: 0.2741 - val_acc: 0.9027
Epoch 62/300
4150/4150 [==============================] - 0s 69us/step - loss: 0.2992 - acc: 0.8865 - val_loss: 0.3556 - val_acc: 0.8613
Epoch 63/300
4150/4150 [==============================] - 0s 69us/step - loss: 0.2958 - acc: 0.8918 - val_loss: 0.2862 - val_acc: 0.8902
Epoch 64/300
4150/4150 [==============================] - 0s 71us/step - loss: 0.2894 - acc: 0.8920 - val_loss: 0.2651 - val_acc: 0.9008
Epoch 65/300
4150/4150 [==============================] - 0s 74us/step - loss: 0.2948 - acc: 0.8892 - val_loss: 0.3529 - val_acc: 0.8622
Epoch 66/300
4150/4150 [==============================] - 0s 71us/step - loss: 0.2983 - acc: 0.8896 - val_loss: 0.2788 - val_acc: 0.8931
Epoch 67/300
4150/4150 [==============================

4150/4150 [==============================] - 0s 68us/step - loss: 0.2279 - acc: 0.9181 - val_loss: 0.2875 - val_acc: 0.8892
Epoch 120/300
4150/4150 [==============================] - 0s 66us/step - loss: 0.2541 - acc: 0.9060 - val_loss: 0.3354 - val_acc: 0.8834
Epoch 121/300
4150/4150 [==============================] - 0s 67us/step - loss: 0.2381 - acc: 0.9106 - val_loss: 0.3002 - val_acc: 0.8911
Epoch 122/300
4150/4150 [==============================] - 0s 70us/step - loss: 0.2597 - acc: 0.9063 - val_loss: 0.3913 - val_acc: 0.8671
Epoch 123/300
4150/4150 [==============================] - 0s 69us/step - loss: 0.2412 - acc: 0.9145 - val_loss: 0.3014 - val_acc: 0.8911
Epoch 124/300
4150/4150 [==============================] - 0s 67us/step - loss: 0.2568 - acc: 0.9058 - val_loss: 0.3298 - val_acc: 0.8873
Epoch 125/300
4150/4150 [==============================] - 0s 69us/step - loss: 0.2348 - acc: 0.9142 - val_loss: 0.3405 - val_acc: 0.8882
Epoch 126/300
4150/4150 [=======================

4150/4150 [==============================] - 0s 71us/step - loss: 0.2278 - acc: 0.9118 - val_loss: 0.2959 - val_acc: 0.8979
Epoch 179/300
4150/4150 [==============================] - 0s 68us/step - loss: 0.2307 - acc: 0.9166 - val_loss: 0.3192 - val_acc: 0.8796
Epoch 180/300
4150/4150 [==============================] - 0s 67us/step - loss: 0.2207 - acc: 0.9178 - val_loss: 0.2865 - val_acc: 0.9046
Epoch 181/300
4150/4150 [==============================] - 0s 68us/step - loss: 0.2380 - acc: 0.9142 - val_loss: 0.3460 - val_acc: 0.8690
Epoch 182/300
4150/4150 [==============================] - 0s 68us/step - loss: 0.2386 - acc: 0.9089 - val_loss: 0.2854 - val_acc: 0.8950
Epoch 183/300
4150/4150 [==============================] - 0s 67us/step - loss: 0.2298 - acc: 0.9116 - val_loss: 0.2896 - val_acc: 0.9046
Epoch 184/300
4150/4150 [==============================] - 0s 71us/step - loss: 0.2249 - acc: 0.9212 - val_loss: 0.2991 - val_acc: 0.8960
Epoch 185/300
4150/4150 [=======================

4150/4150 [==============================] - 0s 92us/step - loss: 0.2362 - acc: 0.9140 - val_loss: 0.3289 - val_acc: 0.8776
Epoch 238/300
4150/4150 [==============================] - 0s 91us/step - loss: 0.2146 - acc: 0.9173 - val_loss: 0.3194 - val_acc: 0.8854
Epoch 239/300
4150/4150 [==============================] - 0s 77us/step - loss: 0.2171 - acc: 0.9181 - val_loss: 0.3668 - val_acc: 0.8699
Epoch 240/300
4150/4150 [==============================] - 0s 72us/step - loss: 0.2521 - acc: 0.9070 - val_loss: 0.2985 - val_acc: 0.8988
Epoch 241/300
4150/4150 [==============================] - 0s 73us/step - loss: 0.2234 - acc: 0.9222 - val_loss: 0.3216 - val_acc: 0.8776
Epoch 242/300
4150/4150 [==============================] - 0s 71us/step - loss: 0.2215 - acc: 0.9193 - val_loss: 0.2817 - val_acc: 0.9066
Epoch 243/300
4150/4150 [==============================] - 0s 71us/step - loss: 0.2169 - acc: 0.9188 - val_loss: 0.2832 - val_acc: 0.8969
Epoch 244/300
4150/4150 [=======================

4150/4150 [==============================] - 0s 74us/step - loss: 0.2012 - acc: 0.9263 - val_loss: 0.2958 - val_acc: 0.8921
Epoch 297/300
4150/4150 [==============================] - 0s 73us/step - loss: 0.2048 - acc: 0.9267 - val_loss: 0.4250 - val_acc: 0.8642
Epoch 298/300
4150/4150 [==============================] - 0s 73us/step - loss: 0.2149 - acc: 0.9210 - val_loss: 0.2951 - val_acc: 0.9037
Epoch 299/300
4150/4150 [==============================] - 0s 74us/step - loss: 0.2193 - acc: 0.9224 - val_loss: 0.2825 - val_acc: 0.8969
Epoch 300/300
4150/4150 [==============================] - 0s 73us/step - loss: 0.2235 - acc: 0.9157 - val_loss: 0.2999 - val_acc: 0.8921


In [13]:
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1,20,11,1)
    return get_labels()[0][np.argmax(model.predict(sample_reshaped))]

print(predict('./DeadSimpleSpeechRecognizer-master/data/bed/016e2c6d_nohash_0.wav', model=model))

bed
